# 01. Data Combination
Combine data from all sources into one **huge** dataframe

In [1]:
# change wd to repo home
import os
if os.getcwd().split('\\')[-1] != 'FM skills prediction':
    os.chdir('..')

In [2]:
import pandas as pd

In [3]:
# load FM data
fm_fd = pd.read_csv('data/FM 2023.csv')

# load FBRef data
fbref_dfs = {filename : pd.read_csv(f'data/FBRef/{filename}', index_col=0)
             for filename in os.listdir('data/FBRef')}

In [34]:
table_types = ['defense', 'gca', 'keepersadv', 'keepers', 'misc', 'passingtypes',
               'passing', 'playingtime', 'possession', 'shooting', 'stats']

fbref_dfs_types = {}

for t in table_types:
    fbref_dfs_types[t] = {}
    for k in fbref_dfs.keys():
        if k.split('_')[1] == t:
            fbref_dfs_types[t][k.split('_')[0]] = fbref_dfs[k]
    fbref_dfs_types[t] = pd.concat(fbref_dfs_types[t], axis=0, ignore_index=True).set_index(['_Player', '_Squad', '_Born'])

In [58]:
fbref_combined_df = pd.concat(fbref_dfs_types, axis=1).reset_index()
fbref_combined_df.columns = fbref_combined_df.columns.map(lambda x: '_'.join(map(str, x)))

In [77]:
fbref_combined_df.to_csv('data/FBRef_combined.csv')